In [2]:
root_data_path = 'E:/AMR/DA/Projekt/data/data_list/0408'

In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
import numpy as np

# === 配置路径 ===
AUDIO_PATH = "E:/AMR/DA/Projekt/data/Audio_files/Call - Great Tit/XC924149.wav"
OUTPUT_CSV = "E:/AMR/DA/Projekt/bird_cls_cnn/data_preprocess/test_demo/birdnet_prediction_result.csv"
OUTPUT_SPEC = "E:/AMR/DA/Projekt/bird_cls_cnn/data_preprocess/test_demo/spec_folder"
os.makedirs(OUTPUT_SPEC, exist_ok=True)

# 提取编号（如 XC638874 或 negative_001）
XC_NUMBER = os.path.splitext(os.path.basename(AUDIO_PATH))[0]

# BirdNET 推理器
analyzer = Analyzer()
rec = Recording(analyzer, AUDIO_PATH)
rec.analyze()

# === 提取推理结果 ===
results = []
for detection in rec.detections:
    start_time = detection["start_time"]
    seg_idx = int(start_time // 3)

    results.append({
        "number": XC_NUMBER,
        "segment_index": seg_idx,
        "start_time": start_time,
        "end_time": detection["end_time"],
        "common_name": detection["common_name"],
        "confidence": detection["confidence"]
    })

# 转为 DataFrame，按 seg 保留最高置信度预测
df = pd.DataFrame(results)
df_top1 = (
    df.sort_values("confidence", ascending=False)
      .groupby(["number", "segment_index"])
      .head(1)
      .reset_index(drop=True)
      .sort_values("segment_index")
)

# === 加载音频用于频谱图生成 ===
y, sr = librosa.load(AUDIO_PATH, sr=48000)

# === 遍历每个保留的 segment，生成频谱图 ===
for _, row in df_top1.iterrows():
    seg_idx = int(row["segment_index"])
    start_sample = seg_idx * 3 * sr
    end_sample = (seg_idx + 1) * 3 * sr
    segment_audio = y[int(start_sample):int(end_sample)]

    # 生成 STFT 幅度谱图
    D = librosa.stft(segment_audio, n_fft=512, hop_length=256)
    S_db = librosa.amplitude_to_db(abs(D), ref=np.max)

    # 画图并保存
    plt.figure(figsize=(6, 3))
    librosa.display.specshow(S_db, sr=sr, hop_length=256, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Spec: {XC_NUMBER} seg {seg_idx}")
    plt.tight_layout()

    save_path = os.path.join(OUTPUT_SPEC, f"{XC_NUMBER}_seg{seg_idx}.png")
    plt.savefig(save_path)
    plt.close()

# === 保存推理结果 CSV ===
df_top1.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 推理完成，预测结果保存至: {OUTPUT_CSV}")
print(f"🖼️ 每段频谱图已保存至: {OUTPUT_SPEC}")



# Bird net批跑

In [ ]:
import os
import pandas as pd
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer

# === 配置路径 ===
CSV_PATH = f"{root_data_path}/train_meta_100_deduplicated.csv"  # 包含 path 列的 CSV
OUTPUT_CSV_SORTED = f"{root_data_path}/birdnet_all_predictions_sorted.csv"

# 初始化 BirdNET Analyzer
analyzer = Analyzer()

# 读取 CSV
df = pd.read_csv(CSV_PATH)

# 存储所有结果
all_results = []

# 遍历每个唯一音频（按 number 分组）
for number in df["number"].unique():
    row = df[df["number"] == number].iloc[0]
    file_path = row["path"]

    if not os.path.exists(file_path):
        print(f"❌ 音频文件不存在: {file_path}")
        continue

    try:
        # 推理
        rec = Recording(analyzer, file_path)
        rec.analyze()

        # 提取预测结果
        detections = rec.detections
        if len(detections) == 0:
            continue

        # 构建结果
        results = []
        for d in detections:
            seg_idx = int(d["start_time"] // 3)
            results.append({
                "number": number,
                "segment_index": seg_idx,
                "start_time": d["start_time"],
                "end_time": d["end_time"],
                "common_name": d["common_name"],
                "confidence": d["confidence"]
            })

        # 每段保留置信度最高的预测
        df_pred = pd.DataFrame(results)
        df_top1 = (
            df_pred.sort_values("confidence", ascending=False)
                   .groupby(["number", "segment_index"])
                   .head(1)
                   .reset_index(drop=True)
        )

        all_results.append(df_top1)
        print(f"✅ 完成推理: {number}, 有效片段数: {len(df_top1)}")

    except Exception as e:
        print(f"⚠️ 推理失败: {file_path}, 错误: {e}")

# 合并结果并排序保存
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df = final_df.sort_values(by=["number", "segment_index"]).reset_index(drop=True)
    final_df.to_csv(OUTPUT_CSV_SORTED, index=False, encoding="utf-8-sig")
    print(f"\n✅ 推理和排序完成，最终结果保存至: {OUTPUT_CSV_SORTED}")
else:
    print("⚠️ 无有效推理结果生成")


# 修正label

In [3]:
import pandas as pd

# === 配置路径 ===
TRAIN_CSV = f"{root_data_path}/all_data_meta_allTypes.csv"
BIRDNET_CSV = f"{root_data_path}/birdnet_all_predictions_sorted.csv"
OUTPUT_CSV = f"{root_data_path}/all_data_meta_allTypes_high_quality.csv"

# === 加载数据 ===
train_df = pd.read_csv(TRAIN_CSV)
birdnet_df = pd.read_csv(BIRDNET_CSV)

# === ✅ 替换名称：将 "Common Blackbird" 统一改为 "Eurasian Blackbird"
train_df["bird_name"] = train_df["bird_name"].replace("Common Blackbird", "Eurasian Blackbird")

# === 统一小写以便对比
train_df["bird_name_lower"] = train_df["bird_name"].str.lower()
birdnet_df["common_name_lower"] = birdnet_df["common_name"].str.lower()

# === 合并两个表格（按 number 和 segment_index 匹配）
merged = train_df.merge(
    birdnet_df,
    how="left",
    on=["number", "segment_index"]
)

# === 条件 1：BirdNET 有结果 且 label 一致
condition1 = (merged["common_name_lower"].notna()) & (merged["bird_name_lower"] == merged["common_name_lower"])

# === 条件 2：标注为 Background Noise（直接保留）
condition2 = (merged["bird_name"] == "Background Noise")

# === 满足任一条件即可保留
cleaned_df = merged[condition1 | condition2]

# === 导出原始字段
cleaned_train_df = cleaned_df[train_df.columns]
cleaned_train_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

# === 输出信息
print(f"✅ 修正后的 train list 已保存至: {OUTPUT_CSV}")
print(f"🎯 保留样本数: {len(cleaned_train_df)} / 原始样本数: {len(train_df)}")
print(f"❌ 舍弃样本数（预测缺失 & 不一致）: {len(train_df) - len(cleaned_train_df)}")


✅ 修正后的 train list 已保存至: E:/AMR/DA/Projekt/data/data_list/0408/all_data_meta_allTypes_high_quality.csv
🎯 保留样本数: 102147 / 原始样本数: 186980
❌ 舍弃样本数（预测缺失 & 不一致）: 84833


# 对比新旧标签

In [4]:
import pandas as pd

# === 配置路径 ===
old_csv_path = f"{root_data_path}/all_data_meta_allTypes.csv"
new_csv_path = f"{root_data_path}/all_data_meta_allTypes_high_quality.csv"

# === 读取 CSV 文件 ===
df_old = pd.read_csv(old_csv_path)
df_new = pd.read_csv(new_csv_path)

# === ✅ 替换类别名称：Common Blackbird → Eurasian Blackbird
df_old["bird_name"] = df_old["bird_name"].replace("Common Blackbird", "Eurasian Blackbird")

# === 获取类别分布 ===
old_counts = df_old["bird_name"].value_counts().sort_index()
new_counts = df_new["bird_name"].value_counts().sort_index()

# === 合并对比表 ===
compare_df = pd.DataFrame({
    "Old Count": old_counts,
    "New Count": new_counts
}).fillna(0).astype(int)

# === 打印类别总数信息 ===
num_classes_old = compare_df["Old Count"].astype(bool).sum()
num_classes_new = compare_df["New Count"].astype(bool).sum()

print(f"\n📊 类别总数（旧数据）: {num_classes_old}")
print(f"📊 类别总数（新数据）: {num_classes_new}")

# === 打印缺失类别（在新数据中为0的）
missing_classes = compare_df[compare_df["New Count"] == 0]
if not missing_classes.empty:
    print(f"\n❌ 以下 {len(missing_classes)} 个类别在精修后被完全剔除：")
    print(missing_classes)
else:
    print("\n✅ 所有类别均有保留")

# === 打印类别数量对比表 ===
print("\n📊 类别数量对比：")
print(compare_df)

# # === 保存对比表 ===
# output_path = "E:/AMR/DA/Projekt/data/data_list/0401/class_distribution_comparison_fixed.csv"
# compare_df.to_csv(output_path, encoding="utf-8-sig")
# print(f"\n✅ 分布对比表已保存至: {output_path}")



📊 类别总数（旧数据）: 33
📊 类别总数（新数据）: 33

✅ 所有类别均有保留

📊 类别数量对比：
                          Old Count  New Count
bird_name                                     
Background Noise               3000       3000
Black-headed Gull              4398       2800
Canada Goose                   4924       3065
Carrion Crow                   5400       1893
Common Chaffinch               7961       3375
Common Kingfisher              3117       1808
Common Redstart                4921       2280
Dunnock                        4307       2854
Eurasian Blackbird             8219       4459
Eurasian Blackcap             10771       4726
Eurasian Blue Tit              5793       3285
Eurasian Bullfinch             4818       3035
Eurasian Coot                  4512       2576
Eurasian Golden Oriole         5066       2980
Eurasian Jay                   9214       2988
Eurasian Nuthatch              5203       3058
Eurasian Siskin                5495       4145
Eurasian Treecreeper           4995       2399
Eura

# 划分训练和测试集

In [5]:
import pandas as pd

# === 配置路径 ===
INPUT_CSV = f"{root_data_path}/all_data_meta_allTypes_high_quality.csv"
TRAIN_CSV = f"{root_data_path}/train_list_high_quality.csv"
TEST_CSV = f"{root_data_path}/valid_list_high_quality.csv"
Valid_Num = 300

# === 读取数据 ===
df = pd.read_csv(INPUT_CSV)

# ✅ 分出 Background Noise 与其他类别
is_bg = df["bird_name"] == "Background Noise"
bg_df = df[is_bg & df["segment_index"].isin([0, 1, 2])].copy()
non_bg_df = df[~is_bg].copy()

# === 初始化容器
train_list = []
test_list = []

# ✅ 处理非 Background Noise 类别
for bird in non_bg_df["bird_name"].unique():
    class_df = non_bg_df[non_bg_df["bird_name"] == bird]
    grouped = class_df.groupby("number")
    groups = list(grouped.groups.items())

    test_samples = []
    used_numbers = set()
    total_test_segments = 0
    target_segments = Valid_Num

    for number, indices in groups:
        group_samples = class_df.loc[indices]
        group_len = len(group_samples)

        # 精确控制测试集大小
        if group_len <= (target_segments - total_test_segments):
            test_samples.append(group_samples)
            used_numbers.add(number)
            total_test_segments += group_len
        if total_test_segments >= target_segments:
            break

    if test_samples:
        test_df = pd.concat(test_samples)
        test_list.append(test_df)
        train_df = class_df[~class_df["number"].isin(used_numbers)]
        train_list.append(train_df)

# ✅ 精准划分 Background Noise（seg0~2）
bg_grouped = bg_df.groupby("number")
bg_test, bg_train = [], []
total_bg_test = 0
target_bg_test = Valid_Num

for number, group in bg_grouped:
    group = group[group["segment_index"].isin([0, 1, 2])]
    group_len = len(group)

    if total_bg_test + group_len <= target_bg_test:
        bg_test.append(group)
        total_bg_test += group_len
    else:
        bg_train.append(group)

# === 合并训练和测试集
final_train = pd.concat(train_list + bg_train).reset_index(drop=True)
final_test = pd.concat(test_list + bg_test).reset_index(drop=True)

# === 保存结果
final_train.to_csv(TRAIN_CSV, index=False, encoding="utf-8-sig")
final_test.to_csv(TEST_CSV, index=False, encoding="utf-8-sig")

print(f"✅ 训练集保存至: {TRAIN_CSV}，共 {len(final_train)} 条")
print(f"✅ 测试集保存至: {TEST_CSV}，共 {len(final_test)} 条")


✅ 训练集保存至: E:/AMR/DA/Projekt/data/data_list/0408/train_list_high_quality.csv，共 92247 条
✅ 测试集保存至: E:/AMR/DA/Projekt/data/data_list/0408/valid_list_high_quality.csv，共 9900 条


In [6]:
import os
import pandas as pd

# === 配置路径 ===
CSV_1_PATH = f"{root_data_path}/train_list_high_quality.csv"
CSV_2_PATH = f"{root_data_path}/valid_list_high_quality.csv"

# === 读取 CSV ===
df1 = pd.read_csv(CSV_1_PATH)
df2 = pd.read_csv(CSV_2_PATH)

# === 提取 bird_name 集合
birds_1 = set(df1["number"].unique())
birds_2 = set(df2["number"].unique())

# === 查找交集
common_birds = birds_1.intersection(birds_2)

# === 打印结果
if common_birds:
    print(f"⚠️ 两个 CSV 存在 {len(common_birds)} 个重复的鸟类：")
    for bird in sorted(common_birds):
        print(" -", bird)
else:
    print("✅ 两个列表中的鸟类完全不重复！")


✅ 两个列表中的鸟类完全不重复！
